In [1]:
#imports
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_internals import MahalanobisMonitor, GaussianMixtureMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor
from monitors_input import SHINE_monitor, SHINE_monitor2
from evaluation import Evaluator

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import utils

ModuleNotFoundError: No module named 'torchvision.models.feature_extraction'

In [ ]:
#model params
batch_size = 10
model = "resnet"

#monitor params
layer_relu_ids = [32]
additional_transform = None
adversarial_attack = None#

#dataset params
id_dataset = "cifar10"
ood_dataset = "svhn"
(id_X_train, id_y_train), (id_X_test, id_y_test) = utils.load_data(id_dataset, None)
(ood_X_train, ood_y_train), (ood_X_test, ood_y_test) = utils.load_data(ood_dataset, None)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
#loading data in pytorch way (for operations with ML models built with pytorch)
tensor_x = torch.Tensor(id_X_train) # transform to torch tensor
tensor_y = torch.Tensor(id_y_train)

my_trainset = TensorDataset(tensor_x,tensor_y)
my_trainset.name = id_dataset
my_trainset.split="train"
my_trainset.network=model
my_trainset.additional_transform=None
my_trainset.adversarial_attack=None
my_trainset.batch_size=batch_size

#test
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_testset = TensorDataset(tensor_x,tensor_y)
my_testset.name = id_dataset
my_testset.split="test"
my_testset.network=model
my_testset.additional_transform=None
my_testset.adversarial_attack=None
my_testset.batch_size=batch_size

#ood
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_dataset_ood = TensorDataset(tensor_x,tensor_y)
my_dataset_ood.name = ood_dataset
my_dataset_ood.split="test"
my_dataset_ood.network=model
my_dataset_ood.additional_transform=None
my_dataset_ood.adversarial_attack=None
my_dataset_ood.batch_size=batch_size

In [ ]:
#extracting and storing features from ID and OOD data using an ML model trained on ID data
feature_extractor = FeatureExtractor(model, id_dataset, layer_relu_ids)

features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(my_trainset)
features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(my_testset)
features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = feature_extractor.get_features(my_dataset_ood)

In [ ]:
#accuracy of the ML model 
id_accuracy = accuracy_score(lab_test, pred_test)
ood_accuracy = 0
if id_dataset == ood_dataset:
    ood_accuracy = accuracy_score(lab_ood, pred_ood)

print("Accuracy")
print("ID:  ", id_accuracy)
print("OOD: ", ood_accuracy)

In [ ]:
id_layer_monitored = -1

In [ ]:
#building SHINE monitor 2 with ID data
monitor_shine_F = SHINE_monitor2(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
monitor_shine_F.fit_by_class(id_X_train, id_y_train, pred_train)

print('number of monitors',len(monitor_shine_F.arr_density))

#building SHINE monitor with ID data
monitor_shine = SHINE_monitor(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
monitor_shine.fit_by_class(id_X_train, id_y_train)

print('number of monitors',len(monitor_shine.arr_density))


In [ ]:
#building oob monitor
monitor_oob = OutsideTheBoxMonitor(n_clusters=3)
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

In [ ]:
#between 0 and 1, percentage of ID data to be preserved
threshold_id = 0.9

In [ ]:
print(np.shape(id_X_test), np.shape(pred_test), np.shape(id_y_test.flatten()))
#testing SHINE  and OOB monitors monitors on ID testset
m_true = []
m_pred = []
m_shine_F = []
m_oob = []

for x, pred, feature, label in tqdm(zip(id_X_test, pred_test, 
                                        features_test[id_layer_monitored], id_y_test.flatten())):
    #shine
    monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
    m_pred.append(monitor_pred)
    
    #shine 2
    monitor_F_pred, pdf = monitor_shine_F.predict(np.array([x]), pred, threshold_id)
    m_shine_F.append(monitor_F_pred)
    
    #oob
    out_box = monitor_oob.predict([feature], [pred])
    m_oob.append(out_box)
    
    if pred == label: #monitor does not need to activate
        m_true.append(0)
    else: #monitor should activate
        m_true.append(1)

In [ ]:
# testing SHINE monitors on entire OOD dataset
#X_ood = np.vstack([ood_X_train, ood_X_test])
#y_ood = np.hstack([ood_y_train, ood_y_test])
print(np.shape(ood_X_test), np.shape(ood_y_test), np.shape(pred_ood))

for x, feature, pred in zip(ood_X_test, features_ood[id_layer_monitored], pred_ood):
    #shine
    monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
    m_pred.append(monitor_pred)
    
    #shine 2
    monitor_F_pred, pdf = monitor_shine_F.predict(np.array([x]), pred, threshold_id)
    m_shine_F.append(monitor_F_pred)
    
    #oob
    out_box = monitor_oob.predict([feature], [pred])
    m_oob.append(out_box)
    
    m_true.append(1) #monitor should always react to novel classes

In [ ]:
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import balanced_accuracy_score

m_true = np.array(m_true)
m_pred = np.array(m_pred).flatten()
m_oob = np.array(m_oob).flatten()


#evaluating SHINE
print('\nSHNE')
print(classification_report(m_true, m_pred))
print(mcc(m_true, m_pred))
print(balanced_accuracy_score(m_true, m_pred))

#evaluating SHINE 2
print('\nSHINE 2')
print(classification_report(m_true, m_shine_F))
print(mcc(m_true, m_shine_F))
print(balanced_accuracy_score(m_true, m_shine_F))

#evaluating OOB
print('\nOOB')
print(classification_report(m_true, m_oob))
print(mcc(m_true, m_oob))
print(balanced_accuracy_score(m_true, m_oob))